# Pedro - Short Queeze Predictor
---

In [9]:
# Initial imports
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [10]:
def create_alpaca_api():
    # Load .env environment variables
    load_dotenv(dotenv_path="Alpaca.env")

    # Set Alpaca API key and secret
    alpaca_api_key = os.getenv("ALPACA_API_KEY")
    alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
    alpaca_api_base_url = "https://paper-api.alpaca.markets"

    # Create the Alpaca API object
    api = tradeapi.REST(
        alpaca_api_key,
        alpaca_secret_key,
        alpaca_api_base_url,
        api_version="v2"
    )

    return api

In [11]:
api = create_alpaca_api()

In [13]:
# Get account information
account = api.get_account()

# Check if our account is restricted from trading.
if account.trading_blocked:
    print('Account is currently restricted from trading.')

# Print our current balance.
print('${} is available as buying power.'.format(account.buying_power))


$200000 is available as buying power.


In [ ]:
SHORT_FLOAT_THRESHOLD = 17
MARKET_CAP_THRESHOLD = 300000000
INSIDER_AMOUNT_THRESHOLD = 500000

In [15]:
# Generate the trading signal 0 or 1,
# where 1 is when short-window (SMA50) is greater than the long (SMA 100)
# and 0 otherwise
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)